In [4]:
import numpy as np
import cv2
from vidstab import VidStab
from os import listdir
import os
import pandas as pd
import matplotlib.pyplot as plt
import skimage 
from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from progressbar import ProgressBar
import seaborn as sns
from os.path import isfile, join
from classifypixelsunet import *
sns.set_style("whitegrid")

Using TensorFlow backend.


In [28]:
def last_4chars(x):
    suf = x.split("_")[1]
    inte = int(suf.split(".")[0])
    return inte

def generate_train_testdata(k):
    """
    generate_train_testdata: Generate table for path to input and output data
                        Args:
                            k: Distance for the next frame
                        Returns:
                             df containg the input path and the corresponding output predicted
    """
    data = pd.DataFrame(columns=["input", "output"])
    directory = "StableFrames/videos"
    subdir = listdir(directory)
    for subd in subdir:
        images = sorted(listdir(directory + "/" + subdir[0]), key=last_4chars)
        for idx, img in enumerate(images):
            base_path = directory + "/" + subd + "/"
            input_img = base_path + img
            output_img = base_path + "image_" + str(idx+k+1) + ".jpg"
            if os.path.isfile(output_img):
                temp_df = pd.DataFrame([[input_img,output_img]], columns=["input", "output"])
                data =  pd.concat((data,temp_df))
    return data    


def get_vid_data(train_test_data, end_idx):
    base_idx = 0
    initial_end_idx = end_idx
    n_vid = int(train_test_data.shape[0]/end_idx)
    new_df = pd.DataFrame()
    for i in range(n_vid):
        print(i)
        new_data = train_test_data.input.iloc[base_idx:end_idx]
        new_df = pd.concat((new_df,new_data), axis = 1)
        base_idx = end_idx
        end_idx = base_idx+initial_end_idx
    cols = ["Video{}".format(x) for x in range(n_vid) ]
    new_df.columns = cols
    return new_df

def read_data(X):
    """
    read_from_path: Read images from path, preprocess, and append to numpy data frame.
                Args:
                    X: List containing all the paths
                Returns:
                    data: numpy array containing all of the data
    """
    new_img_size = 500
    data = np.empty(shape = [1,new_img_size,new_img_size])
    pbar = ProgressBar()
    print("Starting Reading and Preprocessing.\n")
    
    for i in pbar(X):
        img = cv2.imread(i, 0)
        img = img.reshape(1,new_img_size, new_img_size)
        data = np.append(data,img, axis = 0)
    print("Done Reading and Preprocessing.\n")
    data = data.reshape(len(X)+1,new_img_size,new_img_size,1)
    return data


def getFrame(sec,path,stable_vid_name,count):
    vidcap = cv2.VideoCapture(stable_vid_name)
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite(path+"/image_"+str(count)+".jpg", image)     # save frame as JPG file
    return hasFrames

def list_files1(directory, extension):
    """
    list_files1: Find files with a certain extension in the directory and return the names in a list
            Args:
                directory: Directory to be searched
                extension: the extension of the files
            Returns:
                List of files with the extension within the directory
    """
    return list(( (directory + f) for f in listdir(directory) if f.endswith('.' + extension))) 
    
def generate_frames_vid():
    """
    generate_frames_vid: Generates frames for every video in listvideos, and save them in folder with
                         name vidName/
                    Args:
                        listvideos: List containing the path to the video files
                    Returns:
                        None
    """
    try:
        os.mkdir("StableFrames", 755)
    except OSError:
        pass
    try:
        os.mkdir("StableFrames/videos", 755)
    except OSError:
        pass
    listvideos = list_files1('videos/','avi')
    for video in listvideos:
        path = "StableFrames/" + video.split('.avi')[0]
        try:
            os.mkdir(path, 755)
        except OSError:
            pass
        stable_vid_name = 'stable_video.avi'
        stabilizer = VidStab(kp_method='DENSE')
        stabilizer.stabilize(input_path=video, output_path=stable_vid_name)
        sec = 0
        frameRate = 0.1 #//it will capture image in each 0.5 second
        count=1
        success = getFrame(sec,path,stable_vid_name,count)
        while success:
            count = count + 1
            sec = sec + frameRate
            sec = round(sec, 2)
            success = getFrame(sec,path,stable_vid_name,count)

In [10]:
## New contains the frames for every video 
generate_frames_vid()
a = generate_train_testdata(1)



In [29]:
new = get_vid_data(a,67)

0
1
2


In [52]:
### get frames for a certain video
vid_idx = 2
vid1 = new.iloc[:,vid_idx].tolist()
vid1_data = read_data(vid1)
vid_name = vid1[0].split("/")[2].split("_")[1] ## Get video name

Starting Reading and Preprocessing.



Done Reading and Preprocessing.



In [53]:
#Get model
m = vid1_data.shape[1]
model = unet_initialize((m,m))

In [54]:
## Classify frames
def get_classify_frames(vid1_data, model):
    data = np.empty(shape = [1,500,500])
    for i in range(vid1_data.shape[0]):
        img = vid1_data[i,:,:,0]
        preds = unet_classify(model, img)
        preds = preds[:,:,2]
        preds = preds.reshape(1,500,500)
        data = np.append(data,preds, axis = 0)
    return data


In [55]:
## Save classified images
def save_imgs(data):
    try:
        os.mkdir("classifyFrames", 755)
    except OSError:
        pass
    print(data.shape)
    for i in range(data.shape[0]):
        plt.figure()
        plt.imshow(data[i,:,:])
        if (i<10):
            plt.savefig("classifyFrames/img_0{}.png".format(i))
        else:
            plt.savefig("classifyFrames/img_{}.png".format(i))
        plt.close()

In [56]:
data = get_classify_frames(vid1_data, model)
save_imgs(data)

(69, 500, 500)


In [57]:

os.system("ffmpeg -r 10 -i classifyFrames/img_%02d.png {}.avi".format(vid_name))

0